In [14]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from Animals import AnimalShelter





###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "animals"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' #image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#HTML image tag in the line below into the app.

app.layout = html.Div([
#   html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    html.Center(html.B(html.H1('Tyler Pease'))),
    html.Center(html.P('Animal Shelter Dashboard')),
    html.Hr(),
    html.Div(
#interactive filtering options.
        dcc.RadioItems(
        id = 'filter-type',
        #filter options for Rescue Type and Preferred Dog Breeds Table
        options = [
            {'label' : 'Water Rescue', 'value' : 'WTR'},
            {'label' : 'Mountain/Wilderness', 'value' : 'MW'},
            {'label' : 'Disaster/Individual Tracking', 'value' : 'DIT'},
            {'label' : 'Reset to unfiltered', 'value' : 'RES'}
            ],
            value = 'RES',
            labelStyle = {'display' : 'inline-block'}
        )

    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
#features for your interactive data table to make it user-friendly
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = "multi",
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
#filter interactive data table with MongoDB queries
        if filter_type == 'WTR':
            df = pd.DataFrame(list(shelter.read({'$and': [{'sex_upon_outcome' : 'Intact Female'},
                                                          {'$or': [
                                                              {'breed' : 'Labrador Retriever Mix'},
                                                              {'breed' : 'Chesa Bay Retr'},
                                                              {'breed' : 'Newfoundland'}]
                                                          },
                                                          {'$and' : [{'age_upon_outcome_in_weeks' : {'$gte' : 26}},
                                                                     {'age_upon_outcome_in_weeks' : {'$lte' : 156}}]
                                                          }]
                                                })))
        
        elif filter_type == 'MW':
              df = pd.DataFrame(list(shelter.read({'$and': [{'sex_upon_outcome': 'Intact Male'},
                                                            {'$or': [
                                                                {'breed' : 'German Shepherd'},
                                                                {'breed' : 'Alaskan Malamute'},
                                                                {'breed' : 'Old English Sheepdog'},
                                                                {'breed' : 'Siberian Husky'},
                                                                {'breed' : 'Rottweiler'}]
                                                            },
                                                            {'$and' : [{'age_upon_outcome_in_weeks' : {'$gte' : 26}},
                                                                       {'age_upon_outcome_in_weeks' : {'$lte' : 156}}]
                                                            }]
                                                  })))
                               
        elif filter_type == 'DIT':
              df = pd.DataFrame(list(shelter.read({'$and': [{'sex_upon_outcome' : 'Intact Male'},
                                                            {'$or' : [
                                                                {'breed' : 'Doberman Pinscher'},
                                                                {'breed' : 'German Shepherd'},
                                                                {'breed' : 'Golden Retriever'},
                                                                {'breed' : 'Bloodhound'},
                                                                {'breed' : 'Rottweiler'}]
                                                            },
                                                            {'$and' : [{'age_upon_outcome_in_weeks' : {'$gte' : 20}},
                                                                       {'age_upon_outcome_in_weeks' : {'$lte' : 300}}]
                                                            }]
                                                  })))
        
        elif filter_type == 'RES':
              df = pd.DataFrame.from_records(shelter.read({}))
                               
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    #code for chart
    dff = pd.DataFrame.from_dict(viewData)
    
    names = dff['breed'].value_counts().keys().tolist()
    values = dff['breed'].value_counts().tolist()
                               
    return [
        dcc.Graph(
            figure = px.pie(
                data_frame = dff,
                values = values,
                names = names,
                color_discrete_sequence = px.colors.sequential.RdBu,
                width = 800,
                height = 500
            )
        )
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('datatable-id', 'selected_rows'),
     Input('datatable-id', 'selected_columns')])
def update_map(viewData, selected_rows, selected_columns):
#code for geolocation chart
    dff = pd.DataFrame.from_dict(viewData)
    if selected_rows == []:
        selected_rows = [0]
    if len(selected_rows) == 1:
        return [
            dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children =[
                dl.TileLayer(id = "base-layer-id"),
                dl.Marker(position = [(dff.iloc[selected_rows[0],13]), (dff.iloc[selected_rows[0], 14])], children =[
                    dl.Tooltip(dff.iloc[selected_rows[0], 4]),
                    dl.Popup([
                        html.H4("Animal Name"),
                        html.P(dff.iloc[selected_rows[0], 9]),
                        html.H4("Sex"),
                        html.P(dff.iloc[selected_rows[0], 12]),
                        html.H4("Breed"),
                        html.P(dff.iloc[selected_rows[0], 4]),
                        html.H4("Age"),
                        html.P(dff.iloc[selected_rows[0], 15])
                    ])
                ])
            ])
        ]



app